## CS640 Final Project Computer Vision Part

In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#from keras.utils.np_utils import *
#from keras import utils as np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Conv2D, MaxPool2D, GlobalMaxPool2D, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
import os
print(tf.__version__)

2.7.0


In [151]:
IM_HEIGHT = 198
IM_WIDTH = 198
BATCH_SIZE = 4
AGE_SAVE_DIR = "age_model_checkpoint/"
RACE_SAVE_DIR = "race_model_checkpoint/"

def delete_invalid_data(df):
    length = len(df)
    drop_list = []
    for i in range(length):
        r = df.iloc[i]
        image_url = r['img_path']
        try:
            img = Image.open(image_url)
        except IOError:
            drop_list.append(i)
    return df.drop(drop_list)

## Race Prediction

In [152]:
df = pd.read_csv('clean_dataset.csv', usecols=[1, 2, 8])
df['race'] = pd.to_numeric(df['race'])
print(len(df))
df = delete_invalid_data(df)
df.reset_index(drop=True, inplace=True)
train_dataset = df.sample(frac=0.8,ignore_index=True)
test_dataset = df.drop(train_dataset.index)
val_dataset = test_dataset.sample(frac=0.5)
test_dataset = test_dataset.drop(val_dataset.index)
val_dataset.reset_index(drop=True, inplace=True)
test_dataset.reset_index(drop=True, inplace=True)
print(len(df))
print(len(train_dataset))
print(len(test_dataset))
print(len(val_dataset))

3051
2976
2381
297
298


In [153]:
def race_gen(df, for_training, picSize, batchSize):
    races, ages, images = [], [], []
    length = len(df)
    while True:
        for i in range(length):
            r = df.iloc[i]
            race, age, image = r['race'], r['age'], r['img_path']
            img = Image.open(image)
            img = img.resize(picSize, Image.BILINEAR)
            img = img.convert('L')
            img = np.array(img) / 255.0
#             races.append(race - 1)
            races.append(to_categorical((race - 1), 4))
#             ages.append(age)
#             ages.append(to_categorical(age, 2))
            images.append(img)
            if len(images) >= batchSize:
                yield np.array(images), np.array(races)
                images, races = [], []
        if not for_training:
            break


In [154]:
# for testing
picSize = (IM_WIDTH, IM_HEIGHT)
validgen = race_gen(val_dataset, True, picSize, BATCH_SIZE)
traingen = race_gen(train_dataset, True, picSize, BATCH_SIZE)

callbacks = [
    ModelCheckpoint("./race_model_checkpoint", monitor='val_loss')
]
# next(testgen)

In [155]:
def conv_block(inp, filters=32, bn=True, pool=True):
    _ = Conv2D(filters=filters, kernel_size=3, activation='relu')(inp)
    if bn:
        _ = BatchNormalization()(_)
    if pool:
        _ = MaxPool2D()(_)
    return _


input_layer = Input(shape=(IM_HEIGHT, IM_WIDTH, 1))
_ = conv_block(input_layer, filters=32, bn=False, pool=False)
_ = conv_block(_, filters=32*2)
_ = conv_block(_, filters=32*3)
_ = conv_block(_, filters=32*4)
_ = conv_block(_, filters=32*5)
_ = conv_block(_, filters=32*6)
bottleneck = GlobalMaxPool2D()(_)

_ = Dense(units=128, activation='relu')(bottleneck)
race_output = Dense(units=4, activation='softmax', name='race_output')(_)

model = Model(inputs=input_layer, outputs=race_output)
model.compile(optimizer='rmsprop', 
              loss={ 'race_output': 'categorical_crossentropy'},
              metrics={'race_output': 'accuracy'})
model.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 198, 198, 1)]     0         
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 196, 196, 32)      320       
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 194, 194, 64)      18496     
_________________________________________________________________
batch_normalization_75 (Batc (None, 194, 194, 64)      256       
_________________________________________________________________
max_pooling2d_75 (MaxPooling (None, 97, 97, 64)        0         
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 95, 95, 96)        55392     
_________________________________________________________________
batch_normalization_76 (Batc (None, 95, 95, 96)        384

In [156]:
history = model.fit_generator(traingen,
                    steps_per_epoch=len(train_dataset)//batchSize,
                    epochs=10,
                    callbacks=callbacks,
                    validation_data=validgen,
                    validation_steps=len(val_dataset)//batchSize)

C:\Users\DELL\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
595/595 [==============================] - 202s 338ms/step - loss: 1.1730 - accuracy: 0.7371 - val_loss: 1.1874 - val_accuracy: 0.7770
INFO:tensorflow:Assets written to: .\race_model_checkpoint\assets
Epoch 2/10
595/595 [==============================] - 224s 377ms/step - loss: 0.8170 - accuracy: 0.7827 - val_loss: 1.0543 - val_accuracy: 0.7736
INFO:tensorflow:Assets written to: .\race_model_checkpoint\assets
Epoch 3/10
595/595 [==============================] - 223s 375ms/step - loss: 0.8359 - accuracy: 0.7926 - val_loss: 0.9396 - val_accuracy: 0.7736
INFO:tensorflow:Assets written to: .\race_model_checkpoint\assets
Epoch 4/10
595/595 [==============================] - 200s 336ms/step - loss: 0.7924 - accuracy: 0.8007 - val_loss: 0.9235 - val_accuracy: 0.7703
INFO:tensorflow:Assets written to: .\race_model_checkpoint\assets
Epoch 5/10
595/595 [==============================] - 216s 363ms/step - loss: 0.8035 - accuracy: 0.7966 - val_loss: 0.9948 - val_accuracy: 0.7703
INFO:t

In [157]:
testgen = race_gen(test_dataset, False, picSize, 128)
dict(zip(model.metrics_names, model.evaluate_generator(testgen, steps=len(test_dataset)//128)))

C:\Users\DELL\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


{'loss': 0.8351134061813354, 'accuracy': 0.80859375}

In [158]:
testgen = race_gen(test_dataset, False, picSize, 128)
x_test, race_true= next(testgen)
race_pred = model.predict_on_batch(x_test)
race_true = race_true.argmax(axis=-1)
race_pred = race_pred.argmax(axis=-1)
print("Classification report for race")
print(classification_report(race_true, race_pred))

Classification report for race
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.00      0.00      0.00        11
           2       0.00      0.00      0.00         5
           3       0.81      1.00      0.90       104

    accuracy                           0.81       128
   macro avg       0.20      0.25      0.22       128
weighted avg       0.66      0.81      0.73       128



C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# age prediction

In [160]:
df_age = pd.read_csv('clean_dataset_1145.csv', usecols=[0, 6])
df_age['age'] = pd.to_numeric(df_age['age'])
print(len(df_age))
df_age = delete_invalid_data(df_age)
df_age.reset_index(drop=True, inplace=True)
train_dataset = df_age.sample(frac=0.8,ignore_index=True)
test_dataset = df_age.drop(train_dataset.index)
val_dataset = test_dataset.sample(frac=0.5)
test_dataset = test_dataset.drop(val_dataset.index)
val_dataset.reset_index(drop=True, inplace=True)
test_dataset.reset_index(drop=True, inplace=True)
print(len(df_age))
print(len(train_dataset))
print(len(test_dataset))
print(len(val_dataset))

1041
1039
831
104
104


In [161]:
def age_gen(df, for_training, picSize, batchSize):
    ages, images = [], []
    length = len(df)
    while True:
        for i in range(length):
            r = df.iloc[i]
            age, image = r['age'], r['img_path']
            img = Image.open(image)
            img = img.resize(picSize, Image.BILINEAR)
            img = img.convert('L')
            img = np.array(img) / 255.0
#             ages.append(age)
            ages.append(to_categorical(age, 2))
            images.append(img)
            if len(images) >= batchSize:
                yield np.array(images), np.array(ages)
                images, ages = [], []
        if not for_training:
            break

In [162]:
picSize = (IM_WIDTH, IM_HEIGHT)
validgen = age_gen(val_dataset, True, picSize, BATCH_SIZE)
traingen = age_gen(train_dataset, True, picSize, BATCH_SIZE)
filepath = "model_{epoch:02d}-{val_acc:.2f}.hdf5"
callbacks = [
    ModelCheckpoint('./age_model_checkpoint', monitor='val_loss')
]

In [163]:
input_layer = Input(shape=(IM_HEIGHT, IM_WIDTH, 1))
_ = conv_block(input_layer, filters=32, bn=False, pool=False)
_ = conv_block(_, filters=32*2)
_ = conv_block(_, filters=32*3)
_ = conv_block(_, filters=32*4)
_ = conv_block(_, filters=32*5)
_ = conv_block(_, filters=32*6)
bottleneck = GlobalMaxPool2D()(_)

_ = Dense(units=128, activation='relu')(bottleneck)
age_output = Dense(units=2, activation='softmax', name='age_output')(_)

model = Model(inputs=input_layer, outputs=age_output)
model.compile(optimizer='rmsprop', 
              loss={ 'age_output': 'categorical_crossentropy'},
              metrics={'age_output': 'accuracy'})
model.summary()

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 198, 198, 1)]     0         
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 196, 196, 32)      320       
_________________________________________________________________
conv2d_97 (Conv2D)           (None, 194, 194, 64)      18496     
_________________________________________________________________
batch_normalization_80 (Batc (None, 194, 194, 64)      256       
_________________________________________________________________
max_pooling2d_80 (MaxPooling (None, 97, 97, 64)        0         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 95, 95, 96)        55392     
_________________________________________________________________
batch_normalization_81 (Batc (None, 95, 95, 96)        384

In [164]:
history = model.fit_generator(traingen,
                    steps_per_epoch=len(train_dataset)//BATCH_SIZE,
                    epochs=10,
                    callbacks=callbacks,
                    validation_data=validgen,
                    validation_steps=len(val_dataset)//BATCH_SIZE)

C:\Users\DELL\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
207/207 [==============================] - 73s 347ms/step - loss: 1.5015 - accuracy: 0.5992 - val_loss: 1.3617 - val_accuracy: 0.7019
INFO:tensorflow:Assets written to: .\age_model_checkpoint\assets
Epoch 2/10
207/207 [==============================] - 72s 346ms/step - loss: 0.7284 - accuracy: 0.6201 - val_loss: 0.6888 - val_accuracy: 0.6635
INFO:tensorflow:Assets written to: .\age_model_checkpoint\assets
Epoch 3/10
207/207 [==============================] - 74s 359ms/step - loss: 0.6567 - accuracy: 0.6833 - val_loss: 0.6948 - val_accuracy: 0.6923
INFO:tensorflow:Assets written to: .\age_model_checkpoint\assets
Epoch 4/10
207/207 [==============================] - 73s 351ms/step - loss: 0.6621 - accuracy: 0.6799 - val_loss: 0.7025 - val_accuracy: 0.5288
INFO:tensorflow:Assets written to: .\age_model_checkpoint\assets
Epoch 5/10
207/207 [==============================] - 72s 350ms/step - loss: 0.6478 - accuracy: 0.6961 - val_loss: 0.6745 - val_accuracy: 0.6923
INFO:tensorflow

In [165]:
testgen = age_gen(test_dataset, False, picSize, 104)
dict(zip(model.metrics_names, model.evaluate_generator(testgen, steps=len(test_dataset)//104)))

C:\Users\DELL\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


{'loss': 0.9023867249488831, 'accuracy': 0.625}

In [166]:
testgen = age_gen(test_dataset, False, picSize, 104)
x_test, age_true= next(testgen)
age_pred = model.predict_on_batch(x_test)
age_true = age_true.argmax(axis=-1)
age_pred = age_pred.argmax(axis=-1)
print("Classification report for race")
print(classification_report(age_true, age_pred))

Classification report for race
              precision    recall  f1-score   support

           0       0.62      0.98      0.76        64
           1       0.67      0.05      0.09        40

    accuracy                           0.62       104
   macro avg       0.65      0.52      0.43       104
weighted avg       0.64      0.62      0.51       104



In [167]:
df_age.age.value_counts()

0    728
1    311
Name: age, dtype: int64